In [1]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
#df = sqlContext.read.json("s3n://patricks3db/reviews_Movies_and_TV_small.json")
df = sqlContext.read.json("s3n://patricks3db/reviews_Movies_and_TV.json")
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [2]:
#df.take(5)

In [3]:
ratings = df
ratings = ratings.drop("helpful")
ratings = ratings.drop("reviewText")
ratings = ratings.drop("reviewTime")
ratings = ratings.drop("reviewerName")
ratings = ratings.drop("summary")
ratings = ratings.drop("unixReviewTime")
ratings.columns

['asin', 'overall', 'reviewerID']

In [4]:
ratings.persist(StorageLevel.MEMORY_AND_DISK_SER)

DataFrame[asin: string, overall: double, reviewerID: string]

In [5]:
newUserDf = sqlContext.read.json("s3n://patricks3db/mapuidreviewerid/uidreviwerid.json")

In [6]:
newUserDf.take(1)

[Row(reviewerid=u'A1CULZCDCJJGYC', uid=70850)]

In [7]:
newMovieDf = sqlContext.read.json("s3n://patricks3db/mapmidasin/midasin.json")

In [8]:
newMovieDf.take(1)

[Row(asin=u'B00069U0EG', mid=0)]

In [9]:
newMovieDf.persist(StorageLevel.MEMORY_AND_DISK_SER)
newUserDf.persist(StorageLevel.MEMORY_AND_DISK_SER)

DataFrame[reviewerid: string, uid: bigint]

In [10]:
rawRatings1 = ratings.join(newUserDf, ratings.reviewerID == newUserDf.reviewerid)
rawRatings1.printSchema()


root
 |-- asin: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerid: string (nullable = true)
 |-- uid: long (nullable = true)



In [11]:
rawRatings2 = rawRatings1.join(newMovieDf, ratings.asin == newMovieDf.asin)
rawRatings2.printSchema()

root
 |-- asin: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerid: string (nullable = true)
 |-- uid: long (nullable = true)
 |-- asin: string (nullable = true)
 |-- mid: long (nullable = true)



In [12]:
#print rawRatings2.take(1)[0].asin

In [13]:
rawRatings3 = rawRatings2.map(lambda row: (row.uid, row.mid, row.overall)).persist(StorageLevel.MEMORY_AND_DISK_SER)

In [14]:
#rawRatings3.take(10)

In [15]:
#ALS.extractParamMap()

In [16]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
rank = 200
numIterations = 20
#model = ALS.train(rawRatings3, rank, numIterations, 0.01)
#model = ALS.trainImplicit(rawRatings3, rank, numIterations, 0.01)
model = ALS.trainImplicit(rawRatings3, rank, numIterations, 0.03)

In [17]:
model.recommendProducts(100000,5)

[Rating(user=100000, product=181833, rating=0.053093535922371246),
 Rating(user=100000, product=68563, rating=0.04222172255306547),
 Rating(user=100000, product=177592, rating=0.04046698343282806),
 Rating(user=100000, product=38521, rating=0.038640886181047565),
 Rating(user=100000, product=118709, rating=0.028670554914585745)]

In [18]:
#type(model)

In [19]:
#type(ALS)

In [20]:
#userMoviePair = usersDf.join(moviesDf).drop('reviewerID').drop('asin').map(lambda row: (row.userID, row.movieID))

In [21]:
#userMoviePair.take(5)
#type(userMoviePair)

In [22]:
#recommendations = model.predictAll(userMoviePair)

In [23]:
#type(recommendations)

In [24]:
#recommendations.take(5)

In [25]:
path = "s3n://patricks3db/modelsComplete200r20i003a"
model.save(sc, path)

In [26]:
sameModel = MatrixFactorizationModel.load(sc, path)

In [27]:
sameModel.recommendProducts(1,5)

[Rating(user=1, product=57440, rating=0.000292121518318637),
 Rating(user=1, product=145452, rating=0.0002523372938064905),
 Rating(user=1, product=146654, rating=0.00023772202774883556),
 Rating(user=1, product=17539, rating=0.00020044616815766104),
 Rating(user=1, product=142065, rating=0.00019704955938404674)]

In [28]:
sameModel.userFeatures().persist(StorageLevel.MEMORY_AND_DISK_SER)

PythonRDD[531] at RDD at PythonRDD.scala:43

In [29]:
sameModel.recommendProducts(100000,5)

[Rating(user=100000, product=181833, rating=0.053093535922371246),
 Rating(user=100000, product=68563, rating=0.04222172255306547),
 Rating(user=100000, product=177592, rating=0.04046698343282806),
 Rating(user=100000, product=38521, rating=0.038640886181047565),
 Rating(user=100000, product=118709, rating=0.028670554914585745)]

In [30]:
sameModel.recommendProducts(500000,5)

[Rating(user=500000, product=181525, rating=0.024580480290890848),
 Rating(user=500000, product=92296, rating=0.016095680123129952),
 Rating(user=500000, product=107536, rating=0.0047074796159346),
 Rating(user=500000, product=91662, rating=0.003999292888863082),
 Rating(user=500000, product=56600, rating=0.0039477236500175825)]

In [31]:
sameModel.recommendProducts(1000000,5)

[Rating(user=1000000, product=189535, rating=0.008215109228066782),
 Rating(user=1000000, product=24523, rating=0.0061048946107016815),
 Rating(user=1000000, product=118709, rating=0.004881932186930615),
 Rating(user=1000000, product=79542, rating=0.004250134472546344),
 Rating(user=1000000, product=123755, rating=0.004222645851514639)]